# Attention Model
## Author: Gary Corcoran
## Date: Jan. 22nd, 2017

Attention Label Classification for Dashcam Videos

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Dataset Loader

In [2]:
from data_loader import get_loaders

labels_path = '../labels_gary.txt'
data_loaders = get_loaders(labels_path, batch_size=10, num_workers=4)

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(img):
    """
    Imshow for Tensor.
    """
    img = img.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    img = img.astype(np.float32)
    plt.imshow(img)
    
sample_batch = next(iter(data_loaders['Train']))
print(sample_batch['X'].size(), sample_batch['y'].size())

plt.figure(figsize=(20,20))
for i in range(5):
    plt.subplot(5, 5, i*5+1), imshow(sample_batch['X'][i][19])
    plt.xticks([]), plt.yticks([])
    plt.subplot(5, 5, i*5+2), imshow(sample_batch['X'][i][39])
    plt.xticks([]), plt.yticks([])
    plt.subplot(5, 5, i*5+3), imshow(sample_batch['X'][i][59])
    plt.xticks([]), plt.yticks([])
    plt.subplot(5, 5, i*5+4), imshow(sample_batch['X'][i][79])
    plt.xticks([]), plt.yticks([])
    plt.subplot(5, 5, i*5+5), imshow(sample_batch['X'][i][99])
    plt.xticks([]), plt.yticks([])
plt.show()

error: Traceback (most recent call last):
  File "/home/gary/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/gary/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 42, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/gary/pythonWorkspace/AttentionModel/src/data_loader.py", line 50, in __getitem__
    frame = self.transform(frame)
  File "/home/gary/anaconda3/lib/python3.5/site-packages/torchvision-0.2.0-py3.5.egg/torchvision/transforms/transforms.py", line 42, in __call__
    img = t(img)
  File "/home/gary/pythonWorkspace/AttentionModel/src/data_loader.py", line 75, in __call__
    image = cv2.resize(image, self.output_size)
cv2.error: /home/gary/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize



### Load Model

In [ ]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
print(num_ftrs)
model.fc = nn.Linear(num_ftrs, 4)
print(model)

# from model import Network

# net = Network()
# print(net)

### Train Model

In [ ]:
from trainer import train
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

GPU = True
num_epochs = 1
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), learning_rate)

losses, accuracies, best_acc = train(model, data_loaders, GPU, num_epochs, criterion, 
                                     optimizer)

# plot statistics
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.plot(losses['Train'], label='Train'), plt.plot(losses['Valid'], label='Valid')
plt.title('Losses'), plt.xlabel('Epoch'), plt.ylabel('Loss')
plt.legend(loc='lower right')

plt.subplot(122)
plt.plot(accuracies['Train'], label='Train')
plt.plot(accuracies['Valid'], label='Valid')
plt.ylim(0, 100)
plt.title('Accuracy'), plt.xlabel('Epoch'), plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()